In [1]:
import numpy as np
import pandas as pd
import operator

# reading data
orders = pd.read_csv('../data/orders.csv')

# removing all user_ids not in the test set
test  = orders[orders['eval_set'] == 'test' ]
user_ids = test['user_id'].values
orders = orders[orders['user_id'].isin(user_ids)]

all_orders = pd.concat([
    pd.read_csv('../data/order_products__prior.csv', dtype=np.int32, index_col=0),
    pd.read_csv('../data/order_products__train.csv', dtype=np.int32, index_col=0)
]).loc[orders["order_id"]]

/home/joe/kaggle/mercedes/.env/lib/python3.5/site-packages/numpy/lib/arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [2]:
product_info = pd.read_csv("../data/products.csv")

In [3]:
products = pd.DataFrame(
    all_orders.groupby('order_id')['product_id'].apply(list)
)

In [4]:
order_reorders = all_orders.groupby('order_id')[['reordered']].sum()
order_count = all_orders.groupby('order_id')[['reordered']].count().fillna(0)

In [5]:
orders = pd.merge(orders, products, left_on="order_id", right_index=True)

In [6]:
orders = pd.merge(orders, order_count, left_on="order_id", right_index=True)
orders = pd.merge(orders, order_reorders, left_on="order_id", right_index=True)

In [7]:
user_summary = orders.groupby("user_id")[["reordered_x","reordered_y"]].cumsum()
user_reorder_frac = user_summary[["reordered_y"]].div(user_summary["reordered_x"], axis="rows")

In [8]:
user_reorder_frac.columns = ["reorder_frac"]
user_reorder_frac.head()

,reorder_frac
26,0.000000
27,0.157895
28,0.240000
29,0.233333
30,0.341463


In [9]:
orders["last_order"] = orders.groupby("user_id")["order_id"].shift()
orders.head(20)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id,reordered_x,reordered_y,last_order
26,1374495,3,prior,1,1,14,NaN,"[9387.0, 17668.0, 15143.0, 16797.0, 39190.0, 4...",10,0.0,NaN
27,444309,3,prior,2,3,19,9.0,"[38596.0, 21903.0, 248.0, 40604.0, 8021.0, 176...",9,3.0,1374495.0
28,3002854,3,prior,3,3,16,21.0,"[39190.0, 47766.0, 21903.0, 49683.0, 28373.0, ...",6,3.0,444309.0
29,2037211,3,prior,4,2,18,20.0,"[1819.0, 12845.0, 9387.0, 16965.0, 24010.0]",5,1.0,3002854.0
30,2710558,3,prior,5,0,17,12.0,"[39190.0, 9387.0, 17668.0, 47766.0, 16965.0, 2...",11,7.0,2037211.0
31,1972919,3,prior,6,0,16,7.0,"[22035.0, 39190.0, 9387.0, 47766.0, 1819.0, 24...",8,7.0,2710558.0
32,1839752,3,prior,7,0,15,7.0,"[39190.0, 1819.0, 21903.0, 43961.0, 16797.0, 2...",9,7.0,1972919.0
33,3225766,3,prior,8,0,17,7.0,"[39190.0, 22035.0, 43961.0, 18599.0, 21903.0, ...",8,6.0,1839752.0
34,3160850,3,prior,9,0,16,7.0,"[39190.0, 47766.0, 16797.0, 43961.0, 48523.0]",5,5.0,3225766.0
35,676467,3,prior,10,3,16,17.0,"[18599.0, 17668.0, 47766.0, 39190.0, 1005.0, 3...",6,5.0,3160850.0


In [10]:
train = pd.merge(all_orders, orders[orders["eval_set"] != "test"],
                 left_index=True, right_on="last_order")
train = pd.merge(train, user_reorder_frac, how="left", left_on="user_id", right_index=True)
train = pd.merge(train, product_info[["aisle_id", "department_id"]], how="left", left_on="product_id_x", right_index=True)

In [ ]:
train.iloc[::10000]

,product_id_x,add_to_cart_order,reordered,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,product_id_y,reordered_x,reordered_y,last_order,reorder_frac,aisle_id,department_id
27,9387,1.0,0.0,444309,3,prior,2,3,19,9.0,"[38596.0, 21903.0, 248.0, 40604.0, 8021.0, 176...",9,3.0,1.3745e+06,NaN,80.0,11.0
3671,5479,3.0,1.0,3285768,232,prior,8,0,11,4.0,"[8859.0, 18234.0, 6649.0, 45007.0, 48364.0, 40...",14,11.0,942009,NaN,88.0,13.0
6627,10374,1.0,1.0,998305,410,prior,17,0,20,8.0,"[45825.0, 36011.0, 16797.0, 32047.0, 6829.0, 7...",24,18.0,2.10767e+06,0.432000,17.0,13.0
9762,7167,9.0,0.0,1553283,618,prior,8,6,17,12.0,"[39861.0, 41844.0, 49543.0, 2229.0, 22031.0, 3...",6,6.0,2.38361e+06,NaN,121.0,14.0
13304,47042,23.0,0.0,2500149,842,prior,10,1,13,6.0,"[42450.0, 27454.0, 39928.0, 43643.0, 11352.0, ...",21,18.0,2.31883e+06,0.560241,101.0,17.0
16131,11440,19.0,1.0,2563711,999,prior,34,2,15,18.0,"[35168.0, 33195.0, 23087.0, 37029.0, 10753.0, ...",17,9.0,2.71271e+06,NaN,104.0,13.0
19926,34371,7.0,0.0,160976,1238,prior,3,0,22,12.0,"[27662.0, 23909.0, 9076.0, 5450.0, 4006.0, 476...",11,2.0,693862,NaN,81.0,15.0
22656,41751,4.0,1.0,164169,1415,prior,17,3,20,12.0,"[22849.0, 32665.0, 30450.0, 37724.0, 48104.0, ...",11,9.0,474358,0.000000,6.0,2.0
25306,30192,8.0,1.0,221744,1563,prior,47,6,10,3.0,"[21903.0, 15872.0, 24852.0, 47766.0, 47626.0, ...",26,24.0,2.82599e+06,NaN,114.0,17.0
28141,35527,1.0,1.0,1857858,1743,prior,24,1,17,3.0,"[35527.0, 13337.0, 7431.0, 44716.0, 46107.0, 2...",7,2.0,1.97538e+06,NaN,132.0,11.0


In [ ]:
reordered = []
for i, row in train.iterrows():
    reordered.append(row["product_id_x"] in row["product_id_y"])

In [ ]:
train["next_week"] = reordered

In [ ]:
train["dummy"] = 1
train["hour_q"] = train["order_hour_of_day"]//6

In [ ]:
X = train.drop(["last_order", "product_id_y", "product_id_x", "next_week", "reorder_frac",
                "order_id", "user_id", "eval_set", "reordered_x", "reordered_y"], axis=1)
y = train["next_week"]
g = train["order_number"]

In [ ]:
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.model_selection import cross_val_predict, cross_val_score, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Imputer, StandardScaler, MinMaxScaler, PolynomialFeatures
from sklearn.metrics import log_loss, roc_auc_score, f1_score
from xgboost import XGBClassifier

In [ ]:
X["1w"] = (X["days_since_prior_order"] == 7)
X["1m"] = (X["days_since_prior_order"] == 30)

In [ ]:
model = make_pipeline(Imputer(), MinMaxScaler(), LogisticRegression())
                          
preds = cross_val_predict(model,
                          X,
                          #train[["dummy"]],
                          y,
                          method="predict_proba",
                          cv=KFold(shuffle=False),
                         )
print(log_loss(y, preds), roc_auc_score(y, preds[:,1]), f1_score(y, preds[:,1]>0.3))

In [ ]:
model.fit(X,y)

In [ ]:
pd.Series(model.steps[-1][-1].coef_.flatten(), X.columns).plot.bar(figsize=(12,4))

In [ ]:
preds

In [ ]:
model = XGBClassifier(n_estimators=128, learning_rate=0.1, max_depth=3)
xpreds = cross_val_predict(model,
                          X,
                          #train[["dummy"]],
                          y,
                          method="predict_proba",
                          cv=KFold(shuffle=False),
                         )
print(log_loss(y, xpreds), roc_auc_score(y, xpreds[:,1]), f1_score(y, xpreds[:,1]>0.3))
ypreds =( preds +xpreds)/2
print(log_loss(y, ypreds), roc_auc_score(y, ypreds[:,1]), f1_score(y, ypreds[:,1]>0.3))

In [ ]:
model.fit(X,y)

In [ ]:
pd.Series(model.feature_importances_, X.columns).plot.bar(figsize=(12,4))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
preds[:,0] = train["next_week"]
plt.plot(ypreds[99900:100000, 1])
plt.plot(y[99900:100000].values)

## Now generate test predictions

In [ ]:
test = orders[orders["eval_set"] == "test"]

In [ ]:
# for each user, generate a list of the items in the last order
test_products = pd.merge(all_orders, test, left_index=True, right_on="last_order")

In [ ]:
test_products = pd.merge(test_products, user_reorder_frac, how="left", left_on="user_id", right_index=True)
test_products = pd.merge(test_products, product_info[["aisle_id", "department_id"]], how="left", left_on="product_id_x", right_index=True)

In [ ]:
test_products["dummy"] = 1
test_products["hour_q"] = test_products["order_hour_of_day"]//6
test_products["1w"] = (test_products["days_since_prior_order"] == 7)
test_products["1m"] = (test_products["days_since_prior_order"] == 30)

In [ ]:
Xt = test_products.drop(["last_order", "product_id_y", "product_id_x", "reorder_frac",
                         "order_id", "user_id", "eval_set", "reordered_x", "reordered_y"], axis=1)

In [ ]:
model.fit(X, y)

In [ ]:
probs = model.predict_proba(Xt)

In [ ]:
Xt["pred"] = probs[:, 1]

In [ ]:
user_summary.shape

In [ ]:
orders.shape

In [ ]:
test_products["pred"] = probs[:, 1]
test_products.head()

In [ ]:
sub = test_products[test_products["pred"] > 0.3].groupby("order_id")["product_id_x"].apply(lambda x: " ".join(str(int(prod)) for prod in x))

In [ ]:
sub.loc[1528013]